<a href="https://colab.research.google.com/github/divsal009/div/blob/master/Dec6_HOMEAI_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
!pip install fastapi uvicorn pyngrok pandas fuzzywuzzy transformers nest_asyncio


In [38]:
!pip install transformers

In [39]:
!pip install fuzzywuzzy


In [40]:
!pip install flask flask-ngrok fuzzywuzzy transformers


In [41]:
!pip install pyngrok


In [42]:
!pip install flask flask-ngrok pandas fuzzywuzzy transformers


In [43]:
!pip install fastapi uvicorn nest_asyncio pyngrok


In [44]:
!pip install python-Levenshtein


In [45]:
!ngrok config add-authtoken 2pi17RYPIkF3FyQFq4LL2dKzZCi_7c8B2Cxa8bzdg8MK8cayd


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [46]:
!pip install python-multipart


In [47]:
!pip install transformers fastapi uvicorn pandas fuzzywuzzy


In [48]:
!pip install nest_asyncio


In [49]:
import nest_asyncio
nest_asyncio.apply()


In [50]:
!pip install pyngrok


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-269' coro=<Server.serve() done, defined at /usr/local/lib/python3.10/dist-packages/uvicorn/server.py:67> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/main.py", line 579, in run
    server.run()
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/server.py", line 65, in run
    return asyncio.run(self.serve(sockets=sockets))
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "/usr/lib/python3.10/asyncio/tasks.py",

In [51]:
#final

In [ ]:
import os
import pandas as pd
from fuzzywuzzy import fuzz, process
from fastapi import FastAPI, Form
from fastapi.responses import HTMLResponse
from pyngrok import ngrok
import uvicorn
import nest_asyncio
import re
from transformers import pipeline
import nltk
from nltk.corpus import wordnet

# Apply nest_asyncio for compatibility
nest_asyncio.apply()

# Initialize FastAPI app
app = FastAPI()

# Load the Excel data
file_path = "/content/Datasetfinaldivya12_11.xlsx"
if not os.path.exists(file_path):
    raise FileNotFoundError("The dataset file was not found at the specified path.")

# Read Excel Sheets
excel_data = pd.ExcelFile(file_path)
disease_bodyfn_df = excel_data.parse('30-10-DISEASE VS BODYFN')
bodyfn_assistive_df = excel_data.parse('31-10 BODYFN vs assistive techn')
bodyfn_home_icf_df = excel_data.parse('31-10 BODYFN vs HOME ICF')




# Load the Excel data
file_path2 = "/content/Musular_Distrophy19fNOV.xlsx"
if not os.path.exists(file_path2):
    raise FileNotFoundError("The dataset file was not found at the specified path.")

# Read Excel Sheets
excel_data = pd.ExcelFile(file_path2)

# Parse the sheets into DataFrames
md_activity_part_df = excel_data.parse('Activity_Part')
md_housing_adapt_df = excel_data.parse('HouFac_Adapatn')

# Clean the 'Activity_Part' DataFrame by removing unwanted columns
md_activity_part_df = md_activity_part_df.drop(columns=['Unnamed: 2', 'Unnamed: 3'], errors='ignore')

# Clean the column names in Activity_Part DataFrame
md_activity_part_df.columns = md_activity_part_df.columns.str.strip()

# Clean the 'HouFac_Adapatn' DataFrame by stripping extra spaces
md_housing_adapt_df.columns = md_housing_adapt_df.columns.str.strip()

# Rename columns in 'HouFac_Adapatn' to standardize them
md_housing_adapt_df.rename(columns={
    'HOUSING FACTORS': 'Housing Factors',
    'Housing adaptation': 'Housing Adaptation'
}, inplace=True)

# Ensure that you select the correct columns from the cleaned DataFrame
md_activity_part_df = md_activity_part_df[['Disease', 'Activities and participation limitations', 'Assistive Devices: Technologies']]
md_housing_adapt_df = md_housing_adapt_df[['Disease', 'Housing Factors', 'Housing Adaptation']]




# Preprocessing function
def preprocess_text(text1):
    return re.sub(r'[^a-zA-Z0-9\s]', '', text1.strip().lower()) if text1 else ""

# Synonym retrieval function
def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonym = lemma.name().replace('_', ' ')
            if synonym.lower() != word.lower():
                synonyms.add(synonym)
    return synonyms

# Build a precomputed keyword-to-synonym map
def build_keyword_synonym_map(keyword_list):
    synonym_map = {}
    for keyword in keyword_list:
        keyword = preprocess_text(keyword)
        synonym_map[keyword] = get_synonyms(keyword)
    return synonym_map

# Generate keyword map
all_keywords = (
    md_activity_part_df['Activities and participation limitations'].tolist() +
    md_housing_adapt_df['Housing Factors'].tolist()
)
all_keywords = [preprocess_text(k) for k in all_keywords]
keyword_synonym_map = build_keyword_synonym_map(all_keywords)

# Fuzzy Matching Helper Function
def find_closest_match(input_text1, text_list1, threshold=80):
    input_text1 = preprocess_text(input_text1)
    match = process.extractOne(input_text1, text_list1, scorer=fuzz.token_sort_ratio)
    return match[0] if match and match[1] >= threshold else None























# Download NLTK data
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.corpus import stopwords

# Preprocessing helper function
def preprocess_text(text):
    return re.sub(r'[^a-zA-Z0-9\s]', '', text.strip().lower()) if text else ""

# Normalize the dataset
disease_bodyfn_df['Disease'] = disease_bodyfn_df['Disease'].apply(preprocess_text)
bodyfn_assistive_df['Symptoms'] = bodyfn_assistive_df['Symptoms'].apply(preprocess_text)
bodyfn_home_icf_df['Symptoms'] = bodyfn_home_icf_df['Symptoms'].apply(preprocess_text)

# Synonym retrieval function
def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonym = lemma.name().replace('_', ' ')
            if synonym.lower() != word.lower():
                synonyms.add(synonym)
    return synonyms

# Build a precomputed keyword-to-synonym map
def build_keyword_synonym_map(keyword_list):
    synonym_map = {}
    for keyword in keyword_list:
        keyword = preprocess_text(keyword)
        synonym_map[keyword] = get_synonyms(keyword)
    return synonym_map

# Generate the synonym map for all keywords
all_keywords = (
    disease_bodyfn_df['Disease'].tolist()
    + bodyfn_assistive_df['Symptoms'].tolist()
    + bodyfn_home_icf_df['Symptoms'].tolist()
)
all_keywords = [preprocess_text(k) for k in all_keywords]
keyword_synonym_map = build_keyword_synonym_map(all_keywords)

# Fuzzy Matching Helper Function
def find_closest_match(input_text, text_list, threshold=90):
    input_text = preprocess_text(input_text)
    match = process.extractOne(input_text, text_list, scorer=fuzz.token_sort_ratio)
    if match:
        print(f"Attempting to match '{input_text}' - Best match: '{match[0]}' with score: {match[1]}")
    return match[0] if match and match[1] >= threshold else None

# Enhanced Keyword Extraction
def extract_keyword(input_text):
    input_text = preprocess_text(input_text)
    tokens = input_text.split()

    # Break compound words (e.g., "parkinsonsdisease" -> "parkinson's disease")
    compound_breaks = re.findall(r'[a-zA-Z]+', input_text)
    input_text_expanded = ' '.join(compound_breaks)
    tokens_expanded = input_text_expanded.split()

    # Combine all keywords
    all_keywords = (
        disease_bodyfn_df['Disease'].tolist()
        + bodyfn_assistive_df['Symptoms'].tolist()
        + bodyfn_home_icf_df['Symptoms'].tolist()
    )
    all_keywords = [preprocess_text(k) for k in all_keywords]

    # Helper function to score matches
    def score_match(input_tokens, keyword):
        keyword_tokens = keyword.split()
        overlap = set(input_tokens) & set(keyword_tokens)
        return len(overlap)

    # Step 1: Exact match
    exact_match = next((keyword for keyword in all_keywords if keyword == input_text), None)
    if exact_match:
        print(f"Exact match found: {exact_match}")
        return exact_match

    exact_match_expanded = next((keyword for keyword in all_keywords if keyword == input_text_expanded), None)
    if exact_match_expanded:
        print(f"Exact match (expanded) found: {exact_match_expanded}")
        return exact_match_expanded

    # Step 2: Synonym matching with context
    best_synonym_match = None
    best_synonym_score = 0
    for token in tokens + tokens_expanded:
        for keyword, synonyms in keyword_synonym_map.items():
            if token in synonyms or token == keyword:
                score = score_match(tokens, keyword)
                if score > best_synonym_score:
                    best_synonym_match = keyword
                    best_synonym_score = score
    if best_synonym_match:
        print(f"Best synonym match found: {best_synonym_match}")
        return best_synonym_match

    # Step 3: Contextual substring and token matching
    best_context_match = None
    best_context_score = 0
    for keyword in all_keywords:
        score = score_match(tokens + tokens_expanded, keyword)
        if score > best_context_score:
            best_context_match = keyword
            best_context_score = score
    if best_context_match:
        print(f"Contextual substring match found: {best_context_match}")
        return best_context_match

    # Step 4: Fuzzy match
    fuzzy_match = find_closest_match(input_text, all_keywords)
    if fuzzy_match:
        print(f"Fuzzy match found: {fuzzy_match}")
        return fuzzy_match

    fuzzy_match_expanded = find_closest_match(input_text_expanded, all_keywords)
    if fuzzy_match_expanded:
        print(f"Fuzzy match (expanded) found: {fuzzy_match_expanded}")
        return fuzzy_match_expanded

    print(f"No match found for input: {input_text}")
    return None

# Initialize Hugging Face LLM for response generation
generator = pipeline("text-generation", model="distilgpt2")

# Generate conversational response
def generate_conversational_response(keyword, assistive_tech, home_adaptations):
    prompt = (
        f"Provide a detailed and user-friendly explanation about '{keyword}'. "
        f"Assistive technologies include: {assistive_tech}. "
        f"Home adaptations include: {home_adaptations}."
    )
    response = generator(
        prompt,
        max_length=100,
        num_return_sequences=1,
        truncation=True,
    )[0]["generated_text"]
    clean_response = response.split(".")[0] + "."
    return clean_response

# Navbar and styles for web interface
def navbar_and_styles():
    return """
    <style>
        body {
            font-family: Arial, sans-serif;
            background-color: #f0f8ff;
            margin: 0;
            padding: 0;
        }
        .navbar {
            background-color: #004d00;
            padding: 15px;
            text-align: center;
        }
        .navbar a {
            color: white;
            text-decoration: none;
            font-size: 18px;
            margin: 15px;
            font-weight: bold;  /* Makes the link names bold */
        }
        .navbar a:hover {
            text-decoration: underline;
        }
        .container {
            max-width: 800px;
            margin: 50px auto;
            text-align: left;
            padding: 20px;
            background-color: #ffffff;
            border-radius: 10px;
            box-shadow: 0px 4px 10px rgba(0, 0, 0, 0.1);
        }
        h1, h2 {
            color: #004d00;
            margin-bottom: 20px;
        }
        p {
            color: #333;
            font-size: 18px;
            line-height: 1.8;
        }
        input[type="text"] {
            width: 100%;
            padding: 10px;
            font-size: 18px;
            margin: 10px 0;
        }
        button {
            padding: 10px 20px;
            font-size: 18px;
            background-color: #004d00;
            color: white;
            border: none;
            border-radius: 5px;
        }
        button:hover {
            background-color: #003300;
        }
        .result-container {
            padding: 20px;
            background-color: #f9f9f9;
            border-radius: 10px;
            box-shadow: 0 2px 6px rgba(0, 0, 0, 0.1);
        }
        .result-container h2 {
            color: #004d00;
        }
    </style>
    <div class='navbar'>
        <a href="/">Home</a>
        <a href="/disease">Disease/Symptom Search</a>
        <a href="/muscular_dystrophy">Muscular Dystrophy</a>
        <a href="/contact">Contact Us</a>
    </div>
    """

from fastapi import FastAPI
from fastapi.responses import HTMLResponse
from fastapi.staticfiles import StaticFiles

app = FastAPI()

# Serve static files (images, CSS, etc.) from the /content directory
app.mount("/content", StaticFiles(directory="/content"), name="content")

# Home page route


@app.get("/", response_class=HTMLResponse)
async def home():
    return f"""
    <!DOCTYPE html>
    <html>
    <head>
        <title>HOMEAI-ENABLE</title>
        {navbar_and_styles()}
        <style>
            body, html {{
                margin: 0;
                padding: 0;
                height: 100%;
                width: 100%;
                display: flex;
                justify-content: flex-start;
                align-items: flex-start;
                background: url('/content/homaiimage.jpg') no-repeat right bottom;
                background-size: contain; /* Ensure the full image is visible */
                background-position: right bottom; /* Align the image to the bottom right */
                background-attachment: fixed; /* Keeps the image fixed while scrolling */
                text-align: left; /* Ensure text is left aligned */
                padding-top: 60px; /* Create some space between the image and navbar */
            }}

            /* Style for the navbar (green bar) */
            .navbar {{
                padding: 20px;
                background-color: #006400;
                color: white;
                font-weight: bold;
                text-align: center;
                position: fixed;
                width: 100%;
                top: 0;
                left: 0;
                z-index: 10;
            }}

            /* Style for the text content */
            .content {{
                padding: 20px;
                color: darkgreen; /* Dark green text */
                font-weight: bold; /* Bold text */
                font-size: 20px;
                line-height: 1.6;
                max-width: 50%; /* Limit the width of the text */
                margin-top: 20px; /* Add some space above the text */
                word-wrap: break-word; /* Allow the text to wrap */
                white-space: normal; /* Allow text to wrap across multiple lines */
            }}

            h1 {{
                text-align: center;
                font-size: 36px;
            }}

            /* Make sure only 3 words per line */
            .content p {{
                width: 60%; /* Restrict the width of the text block to 60% */
                display: inline-block;
            }}
        </style>
    </head>
    <body>
        <div class="navbar">
            <a href="/" style="color: white; text-decoration: none;">Home</a>
            <a href="/disease" style="color: white; text-decoration: none;">Disease/Symptom Search</a>
            <a href="/muscular_dystrophy" style="color: white; text-decoration: none;">Muscular Dystrophy</a>
            <a href="/contact" style="color: white; text-decoration: none;">Contact Us</a>
        </div>

        <div class="content">
            <h1 style="text-align: left;">HOMEAI-ENABLE</h1>

            <p>Your trusted platform for insights into diseases, symptoms, assistive technologies, and personalized home adaptations.</p>
            <p>Start by entering a disease or symptom, and let us guide you toward solutions that enhance your well-being.</p>
        </div>
    </body>
    </html>
    """



@app.get("/disease", response_class=HTMLResponse)
async def disease_get():
    return f"""
    <!DOCTYPE html>
    <html>
    <head>
        <title>Disease Search</title>
        {navbar_and_styles()}
    </head>
    <body>
        <div class="container">
            <h1>Search for Diseases or Symptoms</h1>
            <form method="post" action="/disease">
                <input type="text" name="disease_name" placeholder="Enter a disease or symptom" required>
                <button type="submit" style="margin-top: 15px;">Search</button>
            </form>
        </div>
    </body>
    </html>
    """

@app.post("/disease", response_class=HTMLResponse)
async def disease_post(disease_name: str = Form(...)):
    keyword = extract_keyword(disease_name)
    response = navbar_and_styles()
    response += "<div class='container'>"

    if not keyword:
        response += f"<h1 style='font-size: 20px;'>Sorry, we couldn't find any information for '{disease_name}'.</h1></div>"
        return HTMLResponse(content=response)

    matched_disease = find_closest_match(keyword, disease_bodyfn_df['Disease'].tolist())
    matched_symptom = find_closest_match(keyword, bodyfn_assistive_df['Symptoms'].tolist() + bodyfn_home_icf_df['Symptoms'].tolist())

    if matched_disease:
        disease_info = disease_bodyfn_df[disease_bodyfn_df['Disease'] == matched_disease]
        symptoms = disease_info['Symptoms'].values[0].split(", ") if not disease_info.empty else []
        response += f"<h1>Here's what I found about the disease '{matched_disease.capitalize()}':</h1>"
        for symptom in symptoms:
            assistive_tech = "No assistive technology found"
            home_adaptations = "No home adaptations found"

            tech_info = bodyfn_assistive_df[bodyfn_assistive_df['Symptoms'] == preprocess_text(symptom)]
            home_info = bodyfn_home_icf_df[bodyfn_home_icf_df['Symptoms'] == preprocess_text(symptom)]

            if not tech_info.empty:
                assistive_tech = tech_info['Assistive Technology'].values[0]
            if not home_info.empty:
                home_adaptations = ", ".join(home_info['Home functions'].dropna().tolist())

            response += f"<div class='result-container'><h2>Symptom: {symptom.capitalize()}</h2><p><strong>Assistive Technologies:</strong> {assistive_tech}</p><p><strong>Home Adaptations:</strong> {home_adaptations}</p></div>"

    if matched_symptom:
        assistive_info = bodyfn_assistive_df[bodyfn_assistive_df['Symptoms'] == matched_symptom]
        home_info = bodyfn_home_icf_df[bodyfn_home_icf_df['Symptoms'] == matched_symptom]

        assistive_tech = assistive_info['Assistive Technology'].values[0] if not assistive_info.empty else "No assistive technology found"
        home_adaptations = ", ".join(home_info['Home functions'].dropna().tolist()) if not home_info.empty else "No home adaptations found"

        response += f"<div class='result-container'><h2>Symptom: {matched_symptom.capitalize()}</h2><p><strong>Assistive Technologies:</strong> {assistive_tech}</p><p><strong>Home Adaptations:</strong> {home_adaptations}</p></div>"

    response += "</div>"
    return HTMLResponse(content=response)






@app.post("/search1", response_class=HTMLResponse)
async def search1(activity_input: str = Form(...), search_type: str = Form(...)):
    text1 = preprocess_text(activity_input)

    if search_type == 'assistive_technology':
        # Check if the input matches any activity
        matching_activities = md_activity_part_df[md_activity_part_df['Activities and participation limitations'].str.contains(text1, case=False, na=False)]

        if not matching_activities.empty:
            response = "<h2>Matching Activities and Assistive Devices:</h2><ul>"
            for _, row in matching_activities.iterrows():
                activity = row['Activities and participation limitations']
                assistive_device = row['Assistive Devices: Technologies']
                response += f"<li><strong>{activity}:</strong> {assistive_device}</li>"
            response += "</ul>"
            return response
        else:
            return f"<h2>No matching activities found for '{activity_input}'</h2>"

    elif search_type == 'housing_adaptation':
        # Check if the input matches any housing factor
        matching_housing = md_housing_adapt_df[md_housing_adapt_df['Housing Factors'].str.contains(text1, case=False, na=False)]

        if not matching_housing.empty:
            response = "<h2>Matching Housing Factors and Adaptations:</h2><ul>"
            for _, row in matching_housing.iterrows():
                housing_factor = row['Housing Factors']
                housing_adaptation = row['Housing Adaptation']
                response += f"<li><strong>{housing_factor}:</strong> {housing_adaptation}</li>"
            response += "</ul>"
            return response
        else:
            return f"<h2>No matching housing factors found for '{activity_input}'</h2>"

    return f"<h2>No matches found for '{activity_input}'</h2>"





@app.get("/muscular_dystrophy", response_class=HTMLResponse)
async def muscular_dystrophy():
    return f"""
    <!DOCTYPE html>
    <html>
    <head>
        <title>Muscular Dystrophy</title>
        {navbar_and_styles()}
        <style>
            /* Removed background color and font style here since navbar_and_styles() will handle them */
            .container {{
                display: flex;
                flex-direction: row;
                justify-content: center;         /* Centers the two columns horizontally */
                align-items: flex-start;         /* Aligns items to the top */
                padding: 20px;
                background: #fff;
                border-radius: 8px;
                box-shadow: 0 0 15px rgba(0, 0, 0, 0.1);
                max-width: 100%;
                margin: 50px auto;
            }}
            .left-column, .right-column {{
                flex: 1;
                padding: 20px;
                background-color: #fff;
                max-width: 50%;
                border-radius: 8px;
                box-shadow: 0 0 5px rgba(0, 0, 0, 0.1);
                text-align: center;
            }}
            h1, h2 {{
                color: #006400;
                text-align: center;
            }}
            p {{
                color: #333;
                font-size: 16px;
                text-align: center;
                max-width: 80%;
                margin: 0 auto;
            }}
            .input-container {{
                margin-bottom: 20px;
                text-align: center;
            }}
            .input-container input[type="text"] {{
                padding: 10px;
                width: 80%;
                border: 1px solid #ccc;
                border-radius: 4px;
            }}
            button {{
                padding: 8px 16px;
                background-color: green;
                color: white;
                border: none;
                border-radius: 4px;
                cursor: pointer;
                margin-top: 10px;
                width: auto;
            }}
            button:hover {{
                background-color: darkgreen;
            }}
            #activities_output, #housing_output {{
                text-align: justify;
                margin: 20px;
                padding: 10px;
                background-color: #f1f1f1;
                border-radius: 8px;
                box-shadow: 0 0 5px rgba(0, 0, 0, 0.1);
            }}
        </style>
    </head>
    <body>

        <div class="container">
            <div class="left-column">
                <h1>Muscular Dystrophy Overview</h1>
                <p>Muscular dystrophy refers to a group of inherited disorders that cause progressive muscle weakness and degeneration.</p>

                <h2>Enter Activities and Participation Limitations:</h2>
                <div class="input-container">
                    <input type="text" id="activities_limitations" placeholder="Enter activities and participation limitations" />
                    <button onclick="searchAssistiveDevices()">Assistive Technologies</button>
                </div>

                <h2>Enter Housing Factors:</h2>
                <div class="input-container">
                    <input type="text" id="housing" placeholder="Enter housing factors" />
                    <button onclick="searchHousing()">Housing Adaptation</button>
                </div>
            </div>

            <div class="right-column">
                <div id="activities_output"></div>
                <div id="housing_output"></div>
            </div>
        </div>

        <script>
            function searchAssistiveDevices() {{
                var inputText = document.getElementById("activities_limitations").value;
                // Clear previous output
                document.getElementById("activities_output").innerHTML = '';
                fetch('/search1', {{
                    method: 'POST',
                    headers: {{ 'Content-Type': 'application/x-www-form-urlencoded' }},
                    body: 'activity_input=' + encodeURIComponent(inputText) + '&search_type=assistive_technology'
                }}).then(response => response.text())
                  .then(data => document.getElementById("activities_output").innerHTML = data);
            }}

            function searchHousing() {{
                var inputText = document.getElementById("housing").value;
                // Clear previous output
                document.getElementById("housing_output").innerHTML = '';
                fetch('/search1', {{
                    method: 'POST',
                    headers: {{ 'Content-Type': 'application/x-www-form-urlencoded' }},
                    body: 'activity_input=' + encodeURIComponent(inputText) + '&search_type=housing_adaptation'
                }}).then(response => response.text())
                  .then(data => document.getElementById("housing_output").innerHTML = data);
            }}
        </script>
    </body>
    </html>
    """




@app.get("/contact", response_class=HTMLResponse)
async def contact():
    return f"""
    <!DOCTYPE html>
    <html>
    <head>
        <title>Contact Us</title>
        {navbar_and_styles()}  <!-- Navbar with styles -->
    </head>
    <body>
        <div class="container">
            <h1>Contact Us</h1>
            <p>If you have any questions or feedback, feel free to reach out to us!</p>
            <p>Email: contact@homeai-enable.com</p>
            <p>Phone: +44 7442685644</p>
        </div>
    </body>
    </html>
    """






# Finally, the line where the FastAPI server is run:
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")
uvicorn.run(app, host="127.0.0.1", port=8000)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Public URL: NgrokTunnel: "https://ef54-34-46-94-253.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [142190]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     81.78.214.135:0 - "GET / HTTP/1.1" 200 OK
INFO:     81.78.214.135:0 - "GET /content/homaiimage.jpg HTTP/1.1" 200 OK
INFO:     81.78.214.135:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
